Here’s the quick, no-fluff tour of what the notebook does:

### 1) Train a grid search on a chosen period

* Downloads daily OHLCV (via yfinance) for your tickers with a warmup buffer.
* Scans **bullish & bearish candlestick patterns** (engulfing, piercing, morning/evening star, harami, hammer/inverted, shooting star, hanging man, doji, dark cloud cover).
* Confirms with toggleable **indicators** (RSI/MACD/ADX/SMA/Bollinger + **Volume** filters).
* **Entry rule:** signal on day *t* → enter at **next day’s open** (*t+1*).
* **Exit rules:** first hit among practical stops (fixed %, ATR multiple, pattern low/high, optional SMA/Donchian trail) or **MAX_HOLD** days.
* **Costs:** Groww-style fees + slippage applied; **₹50k** per trade sizing.
* Evaluates each parameter combo; logs **live** metrics/trades to CSV while running and ranks by an **objective** (default = `total_net`).

### 2) Pick best params and run an out-of-sample backtest

* Takes the **best combo** from the training window.
* Runs a full **OOS** backtest on the test window and saves trades + metrics.

### 3) What you can tweak (top cells)

* **Date ranges:** `TRAIN_START/END`, `TEST_START/END`.
* **Universe:** `TICKERS` list.
* **Side:** `TRADE_SIDE` = `"long"`, `"short"`, or `"both"`.
* **Grid:** `PARAM_GRID` (includes `MIN_CONFIRMATIONS`, `MAX_HOLD`, indicator toggles/values).
* **Stops/costs:** `STOP_RULES`, slippage, fees.
* **Objective:** change `OBJECTIVE` to `win_rate`, `sharpe`, `total_net`, `avg_net` . if you prefer.


### 4) Outputs (organized by period)

```
outputs/grid_v7_train_oos/
  train_<start>_<end>/
    train_summary.csv            # ranked combos with metrics + objective
    train_best_params.json       # winning parameter set
    train_live_metrics.csv       # per-combo metrics as they finish
    train_live_trades.csv        # per-trade rows per combo (live)
    train_best_so_far.json       # rolling best during run
  test_<start>_<end>/
    oos_trades.csv               # trades using best params on OOS
    oos_metrics.json             # summary metrics for OOS
```

### 5) How to run

1. Open the notebook.
2. (Optionally) edit tickers/dates/grid at the top.
3. Run all cells. The last cell performs **Train → OOS** and displays top rows + prints OOS metrics.

If you want, I can switch the ticker list to your full NSE universe in the first cell, or change the objective/stop stack defaults.


In [1]:

import os, json, warnings, math, hashlib, time, itertools
from typing import Dict, Any, List, Optional, Tuple
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
import numpy as np
import pandas as pd
import yfinance as yf
warnings.filterwarnings("ignore")

TICKERS = ['BSE.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BDL.NS', 'BEL.NS', 'BHARTIARTL.NS', 'CHOLAFIN.NS', 'COFORGE.NS', 'DIVISLAB.NS', 'DIXON.NS', 'NYKAA.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'ICICIBANK.NS', 'INDHOTEL.NS', 'INDIGO.NS', 'KOTAKBANK.NS', 'MFSL.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'MUTHOOTFIN.NS', 'PAYTM.NS', 'PERSISTENT.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SRF.NS', 'SHREECEM.NS', 'SOLARINDS.NS', 'TVSMOTOR.NS', 'UNITDSPR.NS']
TRADE_SIDE = "long"
TIMEZONE = "Asia/Kolkata"
EVAL_DATE: Optional[str] = None

TRAIN_START = "2005-01-01"; TRAIN_END = "2015-12-31"
TEST_START  = "2016-01-01"; TEST_END  = "2025-01-01"
WARMUP_DAYS = 400

CAPITAL_PER_TRADE = 50000
MIN_CONFIRMATIONS_DEFAULT = 2
MAX_HOLD_DEFAULT = 10

STOP_RULES = dict(use_fixed_pct=True, fixed_pct=0.05, use_atr=True, atr_len=14, atr_mult=2.0,
                  use_pattern=True, pattern_lookback=1, use_sma_trail=True, sma_trail_len=20,
                  use_donchian=True, donchian_len=20)

USE_COSTS = True
SLIPPAGE_PCT_PER_LEG = 0.0005
BROKERAGE_PER_LEG_RS = 0.0
STT_SELL_PCT = 0.001
EXCHANGE_TXN_PCT = 0.0000345
SEBI_PCT = 0.000001
GST_PCT_ON_BROKERAGE = 0.18
STAMP_DUTY_BUY_PCT = 0.00015

PATTERNS_TO_USE = ["ENGULFING","PIERCING","MORNING_STAR","EVENING_STAR","HARAMI","HARAMI_CROSS",
                   "HAMMER","INVERTED_HAMMER","SHOOTING_STAR","HANGING_MAN","DOJI","DARK_CLOUD_COVER"]

PARAM_GRID = dict(USE_RSI=[True, False], RSI_LEN=[14,21], RSI_LONG_MIN=[50,55], RSI_SHORT_MAX=[50],
                  USE_MACD=[True, False], MACD_FAST=[12], MACD_SLOW=[26], MACD_SIGNAL=[9], MACD_MODE=["hist_above0"],
                  USE_ADX=[True, False], ADX_LEN=[14], ADX_MIN=[18,22],
                  USE_SMA=[True], SMA_FAST=[10,20], SMA_SLOW=[50],
                  USE_BB=[True], BB_LEN=[20], BB_STD=[2.0], BB_SQUEEZE_PCTL=[5,10],
                  USE_VOLUME=[True], VOL_LOOKBACK=[20], VOL_MIN_AVG=[1_000_000, 2_000_000], VOL_MIN_SURGE_RATIO=[1.3,1.5],
                  MIN_CONFIRMATIONS=[2], MAX_HOLD=[7,12,15])


CACHE_DIR = "cache"
os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(os.path.join(CACHE_DIR,"ohlc_range"), exist_ok=True)
os.makedirs(os.path.join(CACHE_DIR,"indicators"), exist_ok=True)
CACHE_MAX_AGE_DAYS = 5
CACHE_INDICATORS = False
MAX_WORKERS_DOWNLOAD = 8
MAX_WORKERS_BACKTEST = 8
OUT_ROOT = "outputs/grid_v7_train_oos"
os.makedirs(OUT_ROOT, exist_ok=True)
print("Chunk1 loaded")


Chunk1 loaded


In [2]:

def _coerce_ohlcv_1d(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for name in ["Open","High","Low","Close","Adj Close","Volume"]:
        if name in out.columns:
            mask = (out.columns == name)
            if getattr(mask, "sum", lambda: 0)() > 1:
                out[name] = out.loc[:, mask].iloc[:, 0]
            else:
                col = out[name]
                out[name] = col.iloc[:, 0] if hasattr(col, "ndim") and col.ndim == 2 else col
    return out

def _dedup_cols(df: pd.DataFrame) -> pd.DataFrame:
    if getattr(df.columns, "duplicated", None) is not None and df.columns.duplicated().any():
        df = df.loc[:, ~df.columns.duplicated()].copy()
    return df

def _get_series(df: pd.DataFrame, name: str):
    if name not in df.columns: return None
    obj = df[name]
    ser = obj.iloc[:,0] if isinstance(obj, pd.DataFrame) else obj
    return ser.reindex(df.index)

def _first_series(df: pd.DataFrame, name_options):
    for nm in name_options:
        if nm in df.columns:
            obj = df.loc[:, df.columns==nm] if (df.columns==nm).sum() > 1 else df[nm]
            s = obj.iloc[:,0] if hasattr(obj,"ndim") and obj.ndim == 2 else obj
            return pd.to_numeric(s, errors="coerce")
    raise KeyError(f"None of {name_options} found")

def _as_bool_series(x, index):
    if isinstance(x, pd.Series):   return x.astype(bool).reindex(index)
    if isinstance(x, pd.DataFrame):return x.iloc[:,0].astype(bool).reindex(index)
    return pd.Series(x, index=index).astype(bool)

def _and_all(series_list, index):
    if not series_list:
        return pd.Series(True, index=index)
    out = _as_bool_series(series_list[0], index)
    for s in series_list[1:]:
        out = out & _as_bool_series(s, index)
    return out

def _scalar_at(df: pd.DataFrame, col: str, i: int) -> float:
    if col not in df.columns: raise KeyError(col)
    obj = df[col]
    if isinstance(obj, pd.DataFrame): obj = obj.iloc[:, 0]
    val = obj.iloc[i]
    try:
        return float(val)
    except Exception:
        if isinstance(val, (list, tuple, np.ndarray)) and len(val) > 0:
            return float(val[0])
        return float(pd.to_numeric(pd.Series([val]), errors="coerce").iloc[0])
print("Chunk2 loaded")


Chunk2 loaded


In [3]:

def sma(series, n): return series.rolling(n, min_periods=n).mean()
def ema(series, n): return series.ewm(span=n, adjust=False).mean()
def rsi(series, n=14):
    s = pd.to_numeric(series.iloc[:,0], errors="coerce") if isinstance(series, pd.DataFrame) else pd.to_numeric(series, errors="coerce")
    delta = s.diff()
    gain = delta.clip(lower=0.0); loss = (-delta).clip(lower=0.0)
    roll_up = gain.rolling(n, min_periods=n).mean(); roll_down = loss.rolling(n, min_periods=n).mean()
    rs = roll_up / roll_down.replace(0, np.nan)
    return (100.0 - (100.0 / (1.0 + rs))).fillna(0.0)

def true_range(df: pd.DataFrame) -> pd.Series:
    H = _first_series(df, ["High","high"]); L = _first_series(df, ["Low","low"]); C = _first_series(df, ["Close","Adj Close","close","adjclose"])
    prev_close = C.shift(1)
    return pd.concat([ (H-L).abs(), (H-prev_close).abs(), (L-prev_close).abs() ], axis=1).max(axis=1)

def atr(df: pd.DataFrame, n=14): return true_range(df).rolling(n, min_periods=n).mean()

def adx(df: pd.DataFrame, n=14):
    H = _first_series(df, ["High","high"]); L = _first_series(df, ["Low","low"]); C = _first_series(df, ["Close","Adj Close","close","adjclose"])
    up_move   = H.diff(); down_move = -L.diff()
    plus_dm  = up_move.clip(lower=0.0); minus_dm = down_move.clip(lower=0.0)
    plus_dm  = plus_dm.where(plus_dm >= minus_dm, 0.0); minus_dm = minus_dm.where(minus_dm > plus_dm, 0.0)
    tr = true_range(pd.DataFrame({"High": H, "Low": L, "Close": C}))
    atr_v = tr.rolling(n, min_periods=n).mean()
    plus_di  = 100.0 * (plus_dm.rolling(n, min_periods=n).mean()  / atr_v)
    minus_di = 100.0 * (minus_dm.rolling(n, min_periods=n).mean() / atr_v)
    denom = (plus_di + minus_di).replace(0, np.nan)
    dx = ((plus_di - minus_di).abs() / denom) * 100.0
    adx_v = dx.rolling(n, min_periods=n).mean()
    return adx_v.fillna(0.0), plus_di.fillna(0.0), minus_di.fillna(0.0)

def pattern_flags(df: pd.DataFrame, patterns):
    O = _first_series(df, ["Open","open"]); H = _first_series(df, ["High","high"])
    L = _first_series(df, ["Low","low"]);   C = _first_series(df, ["Close","Adj Close","close","adjclose"])
    def is_bullish(c,o): return c > o
    def is_bearish(c,o): return c < o
    def body(c,o): return (c-o).abs()
    def range_(h,l): return (h-l).abs()
    rng = range_(H,L); bod = body(C,O)
    small_body = bod <= (rng * 0.3)
    long_lower_shadow = (O - L).abs().where(C>=O, (C - L).abs()) >= rng*0.5
    long_upper_shadow = (H - O).abs().where(C>=O, (H - C).abs()) >= rng*0.5
    flags = {}
    prev_O, prev_H, prev_L, prev_C = O.shift(1), H.shift(1), L.shift(1), C.shift(1)
    if "ENGULFING" in patterns:
        bull = (is_bullish(C,O) & is_bearish(prev_C, prev_O) & (C >= prev_O) & (O <= prev_C))
        bear = (is_bearish(C,O) & is_bullish(prev_C, prev_O) & (C <= prev_O) & (O >= prev_C))
        flags['ENGULFING_BULL'] = bull.fillna(False); flags['ENGULFING_BEAR'] = bear.fillna(False)
    if "PIERCING" in patterns or "DARK_CLOUD_COVER" in patterns:
        prev_mid = (prev_O + prev_C)/2
        flags['PIERCING_BULL'] = (is_bearish(prev_C, prev_O) & is_bullish(C,O) & (O < prev_L) & (C > prev_mid) & (C < prev_O)).fillna(False)
        flags['DARK_CLOUD_COVER_BEAR'] = (is_bullish(prev_C, prev_O) & is_bearish(C,O) & (O > prev_H) & (C < prev_mid) & (C > prev_C)).fillna(False)
    if "MORNING_STAR" in patterns or "EVENING_STAR" in patterns:
        p2_O, p2_H, p2_L, p2_C = O.shift(2), H.shift(2), L.shift(2), C.shift(2)
        flags['MORNING_STAR_BULL'] = (is_bearish(prev_C, prev_O) & (small_body.shift(1)) & (prev_L < p2_L) & is_bullish(C,O) & (C > pd.concat([p2_O,p2_C], axis=1).min(axis=1))).fillna(False)
        flags['EVENING_STAR_BEAR'] = (is_bullish(prev_C, prev_O) & (small_body.shift(1)) & (prev_H > p2_H) & is_bearish(C,O) & (C < pd.concat([p2_O,p2_C], axis=1).max(axis=1))).fillna(False)
    if "HARAMI" in patterns or "HARAMI_CROSS" in patterns:
        prev_min = pd.concat([prev_O, prev_C], axis=1).min(axis=1); prev_max = pd.concat([prev_O, prev_C], axis=1).max(axis=1)
        inside = (O >= prev_min) & (O <= prev_max) & (C >= prev_min) & (C <= prev_max)
        flags['HARAMI_BULL'] = (is_bullish(C,O) & is_bearish(prev_C, prev_O) & inside).fillna(False)
        flags['HARAMI_BEAR'] = (is_bearish(C,O) & is_bullish(prev_C, prev_O) & inside).fillna(False)
        doji = bod <= (rng * 0.1)
        flags['HARAMI_CROSS_BULL'] = (doji & is_bearish(prev_C, prev_O) & inside).fillna(False)
        flags['HARAMI_CROSS_BEAR'] = (doji & is_bullish(prev_C, prev_O) & inside).fillna(False)
    if "HAMMER" in patterns: flags['HAMMER_BULL'] = (long_lower_shadow & (~long_upper_shadow) & (small_body)).fillna(False)
    if "HANGING_MAN" in patterns: flags['HANGING_MAN_BEAR'] = flags['HAMMER_BULL']
    if "INVERTED_HAMMER" in patterns: flags['INVERTED_HAMMER_BULL'] = (long_upper_shadow & (~long_lower_shadow) & (small_body)).fillna(False)
    if "SHOOTING_STAR" in patterns: flags['SHOOTING_STAR_BEAR'] = flags['INVERTED_HAMMER_BULL']
    if "DOJI" in patterns: flags['DOJI_NEUTRAL'] = (bod <= (rng * 0.05)).fillna(False)
    return pd.DataFrame(flags, index=O.index).fillna(False)
print("Chunk3 loaded")


Chunk3 loaded


In [4]:

def add_precomputed_indicators(df: pd.DataFrame, requirements: Dict[str, Any], stop_rules: Dict[str, Any]) -> pd.DataFrame:
    df = _dedup_cols(_coerce_ohlcv_1d(df.copy()))
    close = _first_series(df, ["Close","Adj Close","close","adjclose"])
    high  = _first_series(df, ["High","high"]); low = _first_series(df, ["Low","low"])
    try: vol = _first_series(df, ["Volume","volume"])
    except KeyError: vol = pd.Series(np.nan, index=df.index)
    df["Close"]=close; df["High"]=high; df["Low"]=low; 
    if "Volume" in df.columns or not vol.isna().all(): df["Volume"]=vol
    for n in sorted(requirements.get('RSI_LEN', set())): df[f'RSI_{n}'] = rsi(close, n)
    for (f, s) in sorted(requirements.get('MACD_FS', set())):
        ml = ema(close, f) - ema(close, s); df[f'MACD_LINE_{f}_{s}'] = ml
        for sig in sorted(requirements.get('MACD_SIGNAL', set())):
            ms = ema(ml, sig); df[f'MACD_SIGNAL_{f}_{s}_{sig}'] = ms; df[f'MACD_HIST_{f}_{s}_{sig}'] = ml - ms
    for n in sorted(requirements.get('ADX_LEN', set())):
        adx_v, plus_di, minus_di = adx(df, n); df[f'ADX_{n}']=adx_v; df[f'DI_PLUS_{n}']=plus_di; df[f'DI_MINUS_{n}']=minus_di
    for n in sorted(requirements.get('SMA', set())): df[f'SMA_{n}'] = close.rolling(n, min_periods=n).mean()
    for (n, k) in sorted(requirements.get('BB_NK', set())):
        ma = close.rolling(n, min_periods=n).mean(); sd = close.rolling(n, min_periods=n).std(ddof=0)
        up = ma + k*sd; lo = ma - k*sd; bw = (up - lo) / ma
        df[f'BB_MA_{n}_{k}']=ma; df[f'BB_UP_{n}_{k}']=up; df[f'BB_LO_{n}_{k}']=lo; df[f'BB_BW_{n}_{k}']=bw
    for n in sorted(requirements.get('VOL_LOOKBACK', set())):
        vma = vol.rolling(n, min_periods=n).mean(); df[f'VOL_MA_{n}']=vma; df[f'VOL_SURGE_{n}']=(vol/vma).replace([np.inf,-np.inf],np.nan)
    if stop_rules.get('use_atr', False):
        ln = int(stop_rules.get('atr_len', 14)); df[f'ATR_{ln}'] = true_range(pd.DataFrame({"High": high, "Low": low, "Close": close})).rolling(ln, min_periods=ln).mean()
    if stop_rules.get('use_sma_trail', False):
        ln = int(stop_rules.get('sma_trail_len', 20)); df[f'SMA_TRAIL_{ln}'] = close.rolling(ln, min_periods=ln).mean()
    if stop_rules.get('use_donchian', False):
        ln = int(stop_rules.get('donchian_len', 20)); df[f'DONCH_H_{ln}'] = high.rolling(ln, min_periods=ln).max(); df[f'DONCH_L_{ln}'] = low.rolling(ln, min_periods=ln).min()
    return _dedup_cols(df)

def collect_requirements(param_grid: Dict[str, List[Any]]) -> Dict[str, Any]:
    req = dict(RSI_LEN=set(), MACD_FS=set(), MACD_SIGNAL=set(), ADX_LEN=set(), SMA=set(), BB_NK=set(), VOL_LOOKBACK=set())
    if any(param_grid.get('USE_RSI', [False])): req['RSI_LEN'] = set(param_grid.get('RSI_LEN', []))
    if any(param_grid.get('USE_MACD', [False])): req['MACD_FS'] = set((f, s) for f in param_grid.get('MACD_FAST', []) for s in param_grid.get('MACD_SLOW', [])); req['MACD_SIGNAL'] = set(param_grid.get('MACD_SIGNAL', []))
    if any(param_grid.get('USE_ADX', [False])): req['ADX_LEN'] = set(param_grid.get('ADX_LEN', []))
    if any(param_grid.get('USE_SMA', [False])): req['SMA'] = set(param_grid.get('SMA_FAST', []) + param_grid.get('SMA_SLOW', []))
    if any(param_grid.get('USE_BB', [False])):  req['BB_NK'] = set((n, k) for n in param_grid.get('BB_LEN', []) for k in param_grid.get('BB_STD', []))
    if any(param_grid.get('USE_VOLUME', [False])): req['VOL_LOOKBACK'] = set(param_grid.get('VOL_LOOKBACK', []))
    return req

def req_signature(requirements: Dict[str, Any], stop_rules: Dict[str, Any]) -> str:
    blob = json.dumps({"req": {k: sorted(list(v)) if isinstance(v, set) else v for k, v in sorted(requirements.items())},
                       "stops": {k: stop_rules[k] for k in sorted(stop_rules.keys())}}, sort_keys=True)
    return hashlib.md5(blob.encode()).hexdigest()

def cache_path_ohlc_range(ticker: str, start: str, end: Optional[str]) -> str:
    safe = ticker.replace("/", "_"); end_key = end or "None"
    return os.path.join("cache/ohlc_range", f"{safe}__{start}__{end_key}.pkl")

def is_fresh(path: str, max_age_days: int) -> bool:
    if not os.path.isfile(path): return False
    age_days = (time.time() - os.path.getmtime(path)) / 86400.0
    return age_days <= max_age_days

def download_ohlc_range(ticker: str, start: str, end: Optional[str]) -> pd.DataFrame:
    df = yf.download(ticker, start=start, end=end, interval="1d", progress=False, auto_adjust=True, multi_level_index=False)
    if df is None or df.empty: return pd.DataFrame()
    return df.dropna().copy()

def fetch_ohlc_range_with_cache(ticker: str, start: str, end: Optional[str]) -> pd.DataFrame:
    p = cache_path_ohlc_range(ticker, start, end); os.makedirs(os.path.dirname(p), exist_ok=True)
    if is_fresh(p, CACHE_MAX_AGE_DAYS):
        try: return pd.read_pickle(p)
        except Exception: pass
    df = download_ohlc_range(ticker, start, end)
    if not df.empty:
        try: df.to_pickle(p)
        except Exception: pass
    return df

def fetch_all_ohlc_range(tickers: List[str], start: str, end: Optional[str]) -> Dict[str, pd.DataFrame]:
    out = {}
    with ThreadPoolExecutor(max_workers=MAX_WORKERS_DOWNLOAD) as ex:
        futs = {ex.submit(fetch_ohlc_range_with_cache, t, start, end): t for t in tickers}
        for fut in as_completed(futs):
            t = futs[fut]
            try:
                df = fut.result()
                if not df.empty: out[t] = df
            except Exception as e:
                print("Download error:", t, e)
    return out
print("Chunk4 loaded")


Chunk4 loaded


In [5]:

def indicator_conditions_from_precomp(df: pd.DataFrame, p: dict, side: str) -> dict:
    conds = {}; idx = df.index
    if p.get('USE_RSI', False):
        rs = _get_series(df, f"RSI_{p['RSI_LEN']}"); 
        if rs is not None: conds['RSI'] = (rs >= p['RSI_LONG_MIN']) if side == 'long' else (rs <= p['RSI_SHORT_MAX'])
    if p.get('USE_MACD', False):
        f,s,sg = p['MACD_FAST'], p['MACD_SLOW'], p['MACD_SIGNAL']
        if p['MACD_MODE'] == 'hist_above0':
            h = _get_series(df, f"MACD_HIST_{f}_{s}_{sg}"); 
            if h is not None: conds['MACD'] = (h > 0.0) if side == 'long' else (h < 0.0)
        else:
            l = _get_series(df, f"MACD_LINE_{f}_{s}"); sline = _get_series(df, f"MACD_SIGNAL_{f}_{s}_{sg}")
            if l is not None and sline is not None: conds['MACD'] = (l > sline) if side == 'long' else (l < sline)
    if p.get('USE_ADX', False):
        a = _get_series(df, f"ADX_{p['ADX_LEN']}"); 
        if a is not None: conds['ADX'] = (a >= p['ADX_MIN'])
    if p.get('USE_SMA', False):
        sf, ss = p['SMA_FAST'], p['SMA_SLOW']
        if sf < ss:
            fs = _get_series(df, f"SMA_{sf}"); sl = _get_series(df, f"SMA_{ss}")
            if fs is not None and sl is not None: conds['SMA'] = (fs > sl) if side == 'long' else (fs < sl)
    if p.get('USE_BB', False):
        bn, bk = p['BB_LEN'], p['BB_STD']; bw = _get_series(df, f"BB_BW_{bn}_{bk}")
        if bw is not None:
            thresh = bw.rolling(252, min_periods=252).quantile(p['BB_SQUEEZE_PCTL']/100.0)
            conds['BB'] = (bw <= thresh)
    if p.get('USE_VOLUME', False) and ('Volume' in df.columns or any(c.startswith('VOL_') for c in df.columns)):
        ln = p['VOL_LOOKBACK']; ma = _get_series(df, f"VOL_MA_{ln}"); sur = _get_series(df, f"VOL_SURGE_{ln}")
        clauses = []
        if p.get('VOL_MIN_AVG') is not None and ma is not None: clauses.append(ma >= float(p['VOL_MIN_AVG']))
        if p.get('VOL_MIN_SURGE_RATIO') is not None and sur is not None: clauses.append(sur >= float(p['VOL_MIN_SURGE_RATIO']))
        if clauses: conds['VOL'] = _and_all(clauses, idx)
    for k, v in list(conds.items()): conds[k] = _as_bool_series(v, idx)
    return conds

def indicator_confirmations(df: pd.DataFrame, p: dict, side: str, min_conf: int) -> pd.Series:
    conds = indicator_conditions_from_precomp(df, p, side)
    if not conds: return pd.Series(True, index=df.index)
    stacked = pd.concat([_as_bool_series(v, df.index) for v in conds.values()], axis=1)
    return (stacked.sum(axis=1) >= min_conf).reindex(df.index).fillna(False)

def add_slippage(px: float, side_leg: str) -> float:
    if SLIPPAGE_PCT_PER_LEG <= 0: return px
    return px * (1.0 + SLIPPAGE_PCT_PER_LEG) if side_leg=='buy' else px * (1.0 - SLIPPAGE_PCT_PER_LEG)

def calc_costs(buy_val: float, sell_val: float) -> float:
    if not USE_COSTS: return 0.0
    brokerage_total = BROKERAGE_PER_LEG_RS * 2.0
    exch = EXCHANGE_TXN_PCT * (buy_val + sell_val); sebi = SEBI_PCT * (buy_val + sell_val)
    gst  = GST_PCT_ON_BROKERAGE * (BROKERAGE_PER_LEG_RS * 2.0); stt  = STT_SELL_PCT * sell_val
    stamp = STAMP_DUTY_BUY_PCT * buy_val
    return brokerage_total + exch + sebi + gst + stt + stamp

def compute_initial_stop_levels(df: pd.DataFrame, i_signal: int, side: str, entry_px: float) -> Dict[str, float]:
    levels = {}
    if STOP_RULES.get('use_fixed_pct', False):
        pct = float(STOP_RULES.get('fixed_pct', 0.05))
        levels['fixed'] = entry_px * (1.0 - pct) if side=='long' else entry_px * (1.0 + pct)
    if STOP_RULES.get('use_atr', False):
        ln = int(STOP_RULES.get('atr_len', 14)); mult = float(STOP_RULES.get('atr_mult', 2.0))
        col = f'ATR_{ln}'; atr_val = float(_get_series(df, col).iloc[i_signal]) if col in df else float(atr(df, ln).iloc[i_signal])
        levels['atr'] = entry_px - mult * atr_val if side=='long' else entry_px + mult * atr_val
    if STOP_RULES.get('use_pattern', False):
        lb = max(1, int(STOP_RULES.get('pattern_lookback', 1)))
        lo = df['Low'].iloc[max(0, i_signal - lb + 1):i_signal+1].min(); hi = df['High'].iloc[max(0, i_signal - lb + 1):i_signal+1].max()
        levels['pattern'] = float(lo) if side=='long' else float(hi)
    return levels

def check_trailing_stops(df: pd.DataFrame, j: int, side: str) -> List[Tuple[str, float]]:
    hits = []; H = float(_scalar_at(df, 'High', j)); L = float(_scalar_at(df, 'Low', j))
    if STOP_RULES.get('use_sma_trail', False):
        ln = int(STOP_RULES.get('sma_trail_len', 20)); col = f'SMA_TRAIL_{ln}'
        if col in df and not pd.isna(df[col].iloc[j]):
            s = float(df[col].iloc[j])
            if side=='long' and L <= s: hits.append((f'sma_trail_{ln}', s))
            if side=='short' and H >= s: hits.append((f'sma_trail_{ln}', s))
    if STOP_RULES.get('use_donchian', False):
        ln = int(STOP_RULES.get('donchian_len', 20)); lo_col = f'DONCH_L_{ln}'; hi_col = f'DONCH_H_{ln}'
        lo_ok = (lo_col in df) and (not pd.isna(df[lo_col].iloc[j])); hi_ok = (hi_col in df) and (not pd.isna(df[hi_col].iloc[j]))
        if side=='long' and lo_ok and L <= float(df[lo_col].iloc[j]): hits.append((f'donchian_L_{ln}', float(df[lo_col].iloc[j])))
        if side=='short' and hi_ok and H >= float(df[hi_col].iloc[j]): hits.append((f'donchian_H_{ln}', float(df[hi_col].iloc[j])))
    return hits

def run_backtest_for_symbol(df: pd.DataFrame, params: Dict[str, Any], patterns: List[str], trade_side: str, min_conf: int, max_hold: int):
    df = _dedup_cols(_coerce_ohlcv_1d(df)); pats = pattern_flags(df, patterns)
    bull_cols = [c for c in pats.columns if c.endswith('_BULL')]; bear_cols = [c for c in pats.columns if c.endswith('_BEAR')]
    long_pat  = pats[bull_cols].any(axis=1) if bull_cols else pd.Series(False, index=df.index)
    short_pat = pats[bear_cols].any(axis=1) if bear_cols else pd.Series(False, index=df.index)
    long_conf  = indicator_confirmations(df, params, 'long',  min_conf); short_conf = indicator_confirmations(df, params, 'short', min_conf)
    long_entry  = (long_pat  & long_conf).fillna(False); short_entry = (short_pat & short_conf).fillna(False)
    if trade_side == "long":  short_entry[:] = False
    if trade_side == "short": long_entry[:]  = False
    trades = []; i = 0; N = len(df.index); idx = df.index
    def _ts_full(ts): return pd.Timestamp(ts).strftime("%Y-%m-%d %H:%M:%S")
    def _ts_date(ts): return str(pd.Timestamp(ts).date())
    def _ts_time(ts): return pd.Timestamp(ts).strftime("%H:%M:%S")
    while i < N:
        side = 'long' if long_entry.iloc[i] else ('short' if short_entry.iloc[i] else None)
        if side is None: i += 1; continue
        entry_bar = i + 1
        if entry_bar >= N: break
        signal_ts = idx[i]; entry_ts = idx[entry_bar]
        entry_open = float(_scalar_at(df, 'Open', entry_bar))
        entry_px = add_slippage(entry_open, 'buy' if side=='long' else 'sell')
        qty = int(CAPITAL_PER_TRADE // entry_px)
        if qty <= 0: i += 1; continue
        init_levels = compute_initial_stop_levels(df, i, side, entry_px)
        static_stop = (min(init_levels.values()) if side=='long' else max(init_levels.values())) if init_levels else None
        pats_cols = [c for c in (bull_cols if side=='long' else bear_cols) if pats[c].iloc[i]]
        conds = indicator_conditions_from_precomp(df, params, side)
        conf_true = [name for name, ser in conds.items() if bool(ser.iloc[i])]
        snap_cols = []
        if params.get('USE_RSI', False): snap_cols.append(f"RSI_{params['RSI_LEN']}")
        if params.get('USE_MACD', False):
            f,s,sg = params['MACD_FAST'], params['MACD_SLOW'], params['MACD_SIGNAL']
            snap_cols += [f"MACD_LINE_{f}_{s}", f"MACD_SIGNAL_{f}_{s}_{sg}", f"MACD_HIST_{f}_{s}_{sg}"]
        if params.get('USE_ADX', False): snap_cols.append(f"ADX_{params['ADX_LEN']}")
        if params.get('USE_SMA', False): snap_cols += [f"SMA_{params['SMA_FAST']}", f"SMA_{params['SMA_SLOW']}"]
        if params.get('USE_BB', False):
            bn, bk = params['BB_LEN'], params['BB_STD']; snap_cols.append(f"BB_BW_{bn}_{bk}")
        if params.get('USE_VOLUME', False):
            ln = params['VOL_LOOKBACK']; snap_cols += [f"VOL_MA_{ln}", f"VOL_SURGE_{ln}"]
        if STOP_RULES.get('use_atr', False): ln = int(STOP_RULES.get('atr_len', 14)); snap_cols.append(f'ATR_{ln}')
        if STOP_RULES.get('use_sma_trail', False): ln = int(STOP_RULES.get('sma_trail_len', 20)); snap_cols.append(f'SMA_TRAIL_{ln}')
        snapshot = {}
        for sc in snap_cols:
            if sc in df.columns:
                try:
                    val = _scalar_at(df, sc, i)
                    if not (pd.isna(val) or np.isinf(val)): snapshot[sc] = float(val)
                except Exception: pass
        exit_bar = None; reason = None; j = entry_bar
        while j < N and (j - entry_bar) <= max_hold:
            H = float(_scalar_at(df, 'High', j)); L = float(_scalar_at(df, 'Low', j)); C = float(_scalar_at(df, 'Close', j))
            if static_stop is not None:
                if side=='long' and L <= static_stop: exit_bar, reason, exit_px = j, "SL_STATIC", static_stop; break
                if side=='short' and H >= static_stop: exit_bar, reason, exit_px = j, "SL_STATIC", static_stop; break
            hits = check_trailing_stops(df, j, side)
            if hits: tag, level = hits[0]; exit_bar, reason, exit_px = j, f"SL_{tag.upper()}", level; break
            if (j - entry_bar) >= max_hold: exit_bar, reason, exit_px = j, "MAX_HOLD", C; break
            j += 1
        if exit_bar is None: exit_bar, reason = N - 1, "FORCE_LAST"; exit_px = float(_scalar_at(df, 'Close', exit_bar))
        exit_ts = idx[exit_bar]
        buy_val  = entry_px * qty if side=='long' else exit_px * qty
        sell_val = exit_px * qty if side=='long' else entry_px * qty
        fees = calc_costs(buy_val, sell_val) if USE_COSTS else 0.0
        gross = (exit_px - entry_px) * qty if side=='long' else (entry_px - exit_px) * qty
        net   = gross - fees
        trades.append(dict(signal_dt=_ts_full(signal_ts), entry_dt=_ts_full(entry_ts), entry_time=_ts_time(entry_ts),
                           exit_dt=_ts_full(exit_ts), exit_time=_ts_time(exit_ts),
                           entry_date=_ts_date(entry_ts), exit_date=_ts_date(exit_ts),
                           pattern_detected=",".join(pats_cols), exit_reason=reason,
                           side=side, entry_px=entry_px, exit_px=exit_px, qty=qty, static_stop=static_stop,
                           gross_pnl=gross, fees=fees, net_pnl=net, reason=reason,
                           signal_bar=_ts_date(signal_ts), patterns=",".join(pats_cols)))
        i = exit_bar + 1
    tdf = pd.DataFrame(trades); 
    # Fill required keys if missing
    if 'indicator_conf_true' not in tdf.columns: tdf['indicator_conf_true']=""
    if 'indicator_values' not in tdf.columns: tdf['indicator_values']=""
    if 'stop_levels' not in tdf.columns: tdf['stop_levels']=""
    pnl = tdf['net_pnl'].values if not tdf.empty else np.array([])
    wins = (pnl > 0).sum() if pnl.size else 0
    wr = (wins / pnl.size) if pnl.size else 0.0
    avg = float(np.mean(pnl)) if pnl.size else 0.0
    total = float(np.sum(pnl)) if pnl.size else 0.0
    eq = np.cumsum(pnl) if pnl.size else np.array([])
    peak = np.maximum.accumulate(eq) if eq.size else np.array([])
    dd = (peak - eq) if eq.size else np.array([])
    max_dd = float(np.max(dd)) if dd.size else 0.0
    sd = float(np.std(pnl)) if pnl.size else 0.0
    sharpe = (avg / sd) if sd > 1e-9 else 0.0
    metrics = dict(trades=int(pnl.size), win_rate=wr, avg_net=avg, total_net=total, max_dd=max_dd, sharpe=sharpe)
    return tdf, metrics

OBJECTIVE = "total_net"
def objective_value(metrics: Dict[str, Any]) -> float: return float(metrics.get(OBJECTIVE, 0.0))

def expand_grid(grid: Dict[str, List[Any]]) -> List[Dict[str, Any]]:
    keys = list(grid.keys()); vals = [grid[k] for k in keys]; combos = []
    for prod in itertools.product(*vals):
        p = dict(zip(keys, prod))
        if p.get('USE_SMA', False) and p['SMA_FAST'] >= p['SMA_SLOW']: continue
        combos.append(p)
    return combos

def run_backtest_universe_for_params(ticker_to_df: Dict[str, pd.DataFrame], params: Dict[str, Any], trade_side: str):
    min_conf = int(params.get('MIN_CONFIRMATIONS', MIN_CONFIRMATIONS_DEFAULT)); max_hold = int(params.get('MAX_HOLD', MAX_HOLD_DEFAULT))
    all_trades = []
    def worker(t, df):
        tdf, _ = run_backtest_for_symbol(df, params, PATTERNS_TO_USE, trade_side, min_conf, max_hold)
        if not tdf.empty: tdf.insert(0, 'ticker', t)
        return tdf
    with ThreadPoolExecutor(max_workers=MAX_WORKERS_BACKTEST) as ex:
        futs = {ex.submit(worker, t, df): t for t, df in ticker_to_df.items()}
        for fut in as_completed(futs):
            try:
                out = fut.result()
                if out is not None and not out.empty: all_trades.append(out)
            except Exception as e:
                print("Backtest error:", futs[fut], e)
    base_cols = ['ticker','signal_dt','entry_dt','entry_time','exit_dt','exit_time','entry_date','exit_date','pattern_detected','exit_reason','side','entry_px','exit_px','qty','static_stop','gross_pnl','fees','net_pnl','reason','signal_bar','patterns','indicator_conf_true','indicator_values','stop_levels']
    trades_df = pd.concat(all_trades, axis=0, ignore_index=True) if all_trades else pd.DataFrame(columns=base_cols)
    pnl = trades_df['net_pnl'].values if not trades_df.empty else np.array([])
    wins = (pnl > 0).sum() if pnl.size else 0
    wr = (wins / pnl.size) if pnl.size else 0.0
    avg = float(np.mean(pnl)) if pnl.size else 0.0
    total = float(np.sum(pnl)) if pnl.size else 0.0
    eq = np.cumsum(pnl) if pnl.size else np.array([])
    peak = np.maximum.accumulate(eq) if eq.size else np.array([])
    dd = (peak - eq) if eq.size else np.array([])
    max_dd = float(np.max(dd)) if dd.size else 0.0
    sd = float(np.std(pnl)) if pnl.size else 0.0
    sharpe = (avg / sd) if sd > 1e-9 else 0.0
    metrics = dict(trades=int(pnl.size), win_rate=wr, avg_net=avg, total_net=total, max_dd=max_dd, sharpe=sharpe)
    return trades_df, metrics
print("Chunk5 loaded")


Chunk5 loaded


In [6]:

def _safe_json(o):
    try: return json.dumps(o, ensure_ascii=False)
    except Exception: return str(o)

def _append_csv(path: str, row: dict, field_order: list):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    is_new = not os.path.exists(path)
    with open(path, "a", encoding="utf-8") as f:
        if is_new: f.write(",".join(field_order) + "\n")
        def _fmt(v):
            if isinstance(v, str): return '"' + v.replace('"','""') + '"'
            return "" if v is None else str(v)
        f.write(",".join(_fmt(row.get(k, "")) for k in field_order) + "\n")

def fetch_enriched_frames_for_range(tickers: List[str], param_grid: Dict[str, List[Any]], start: str, end: Optional[str], warmup_days: int) -> Dict[str, pd.DataFrame]:
    start_ts = pd.Timestamp(start); fetch_start = (start_ts - pd.Timedelta(days=warmup_days)).strftime("%Y-%m-%d")
    raw = fetch_all_ohlc_range(tickers, fetch_start, end)
    req = collect_requirements(param_grid); enriched_map = {}
    for t, df in raw.items():
        if df.empty: continue
        enriched_map[t] = add_precomputed_indicators(df.copy(), req, STOP_RULES)
    return enriched_map

def slice_frames(frames: Dict[str, pd.DataFrame], start: str, end: Optional[str]) -> Dict[str, pd.DataFrame]:
    start_ts = pd.Timestamp(start); end_ts = pd.Timestamp(end) if end else None
    out = {}
    for t, df in frames.items():
        if df.empty: continue
        di = df.copy(); idx = di.index.normalize()
        mask = (idx >= start_ts.normalize())
        if end_ts is not None: mask &= (idx <= end_ts.normalize())
        out[t] = di.loc[mask].copy()
    return out

def grid_search_on_frames(frames: Dict[str, pd.DataFrame], param_grid: Dict[str, List[Any]], out_dir: str, label: str = "train", live: bool = True):
    combos = expand_grid(param_grid); total = len(combos)
    print(f"[{label}] Total parameter combos: {total}"); os.makedirs(out_dir, exist_ok=True)
    live_metrics_csv = os.path.join(out_dir, f"{label}_live_metrics.csv")
    live_trades_csv  = os.path.join(out_dir, f"{label}_live_trades.csv")
    best_json        = os.path.join(out_dir, f"{label}_best_so_far.json")
    METRIC_FIELDS = ["ts","combo_idx","total_combos","objective","trades","win_rate","avg_net","total_net","max_dd","sharpe","params_json"]
    TRADE_FIELDS  = ["ts","combo_idx","ticker","side","signal_dt","entry_dt","entry_time","exit_dt","exit_time","pattern_detected","exit_reason","entry_px","exit_px","qty","gross_pnl","fees","net_pnl","params_json"]
    results, combo_summaries = [], []; best_obj = -1e308
    for k, p in enumerate(combos, start=1):
        print(f"[{label} {k}/{total}] Params:", p)
        trades_df, metrics = run_backtest_universe_for_params(frames, p, TRADE_SIDE)
        obj = objective_value(metrics); ts_now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        if live:
            mrow = dict(ts=ts_now, combo_idx=k, total_combos=total, objective=round(obj,4), trades=int(metrics.get("trades",0)),
                        win_rate=round(metrics.get("win_rate",0.0),4), avg_net=round(metrics.get("avg_net",0.0),2),
                        total_net=round(metrics.get("total_net",0.0),2), max_dd=round(metrics.get("max_dd",0.0),2),
                        sharpe=round(metrics.get("sharpe",0.0),4), params_json=_safe_json(p))
            _append_csv(live_metrics_csv, mrow, METRIC_FIELDS)
            if trades_df is not None and not trades_df.empty:
                for _, r in trades_df.iterrows():
                    trow = dict(ts=ts_now, combo_idx=k, ticker=r.get("ticker",""), side=r.get("side",""),
                                signal_dt=r.get("signal_dt",""), entry_dt=r.get("entry_dt",""), entry_time=r.get("entry_time",""),
                                exit_dt=r.get("exit_dt",""), exit_time=r.get("exit_time",""),
                                pattern_detected=r.get("pattern_detected", r.get("patterns","")), exit_reason=r.get("exit_reason", r.get("reason","")),
                                entry_px=r.get("entry_px",""), exit_px=r.get("exit_px",""), qty=r.get("qty",""),
                                gross_pnl=r.get("gross_pnl",""), fees=r.get("fees",""), net_pnl=r.get("net_pnl",""),
                                params_json=_safe_json(p))
                    _append_csv(live_trades_csv, trow, TRADE_FIELDS)
        if obj > best_obj:
            best_obj = obj
            with open(best_json, "w", encoding="utf-8") as bf:
                json.dump({"combo_idx": k, "objective": obj, "params": p, "metrics": metrics}, bf, ensure_ascii=False, indent=2)
        results.append((p, trades_df, metrics)); combo_summaries.append(dict(params=p, **metrics))
    summ = pd.DataFrame(combo_summaries)
    if not summ.empty:
        summ['objective'] = summ.apply(lambda r: objective_value(r.to_dict()), axis=1)
        summ = summ.sort_values('objective', ascending=False).reset_index(drop=True)
    summ.to_csv(os.path.join(out_dir, f"{label}_summary.csv"), index=False)
    if not summ.empty:
        best_params = summ.loc[0, 'params']
        with open(os.path.join(out_dir, f"{label}_best_params.json"), "w", encoding="utf-8") as f:
            json.dump(best_params, f, ensure_ascii=False, indent=2)
    else:
        best_params = None
    return summ, results, best_params

def backtest_oos(frames: Dict[str, pd.DataFrame], params: Dict[str, Any], out_dir: str, label: str = "oos"):
    print(f"[{label}] Backtesting with best params:", params)
    trades_df, metrics = run_backtest_universe_for_params(frames, params, TRADE_SIDE)
    os.makedirs(out_dir, exist_ok=True)
    if not trades_df.empty: trades_df.to_csv(os.path.join(out_dir, f"{label}_trades.csv"), index=False)
    with open(os.path.join(out_dir, f"{label}_metrics.json"), "w", encoding="utf-8") as f:
        json.dump(metrics, f, ensure_ascii=False, indent=2)
    return trades_df, metrics

# Runner
train_start, train_end = TRAIN_START, TRAIN_END
test_start,  test_end  = TEST_START, TEST_END
enriched_all = fetch_enriched_frames_for_range(TICKERS, PARAM_GRID, start=train_start, end=test_end, warmup_days=WARMUP_DAYS)
train_frames = slice_frames(enriched_all, train_start, train_end)
test_frames  = slice_frames(enriched_all, test_start, test_end)
train_dir = os.path.join(OUT_ROOT, f"train_{train_start}_{train_end}")
test_dir  = os.path.join(OUT_ROOT, f"test_{test_start}_{test_end}")
os.makedirs(train_dir, exist_ok=True); os.makedirs(test_dir, exist_ok=True)
train_summary, train_results, best_params = grid_search_on_frames(train_frames, PARAM_GRID, out_dir=train_dir, label="train", live=True)
display(train_summary.head(10))
if best_params is not None:
    oos_trades, oos_metrics = backtest_oos(test_frames, best_params, out_dir=test_dir, label="oos")
    print("OOS Metrics:", oos_metrics); display(oos_trades.head())
else:
    print("No best params found on training period.")
print("Chunk6 loaded")


[train] Total parameter combos: 3072
[train 1/3072] Params: {'USE_RSI': True, 'RSI_LEN': 14, 'RSI_LONG_MIN': 50, 'RSI_SHORT_MAX': 50, 'USE_MACD': True, 'MACD_FAST': 12, 'MACD_SLOW': 26, 'MACD_SIGNAL': 9, 'MACD_MODE': 'hist_above0', 'USE_ADX': True, 'ADX_LEN': 14, 'ADX_MIN': 18, 'USE_SMA': True, 'SMA_FAST': 10, 'SMA_SLOW': 50, 'USE_BB': True, 'BB_LEN': 20, 'BB_STD': 2.0, 'BB_SQUEEZE_PCTL': 5, 'USE_VOLUME': True, 'VOL_LOOKBACK': 20, 'VOL_MIN_AVG': 1000000, 'VOL_MIN_SURGE_RATIO': 1.3, 'MIN_CONFIRMATIONS': 2, 'MAX_HOLD': 7}
[train 2/3072] Params: {'USE_RSI': True, 'RSI_LEN': 14, 'RSI_LONG_MIN': 50, 'RSI_SHORT_MAX': 50, 'USE_MACD': True, 'MACD_FAST': 12, 'MACD_SLOW': 26, 'MACD_SIGNAL': 9, 'MACD_MODE': 'hist_above0', 'USE_ADX': True, 'ADX_LEN': 14, 'ADX_MIN': 18, 'USE_SMA': True, 'SMA_FAST': 10, 'SMA_SLOW': 50, 'USE_BB': True, 'BB_LEN': 20, 'BB_STD': 2.0, 'BB_SQUEEZE_PCTL': 5, 'USE_VOLUME': True, 'VOL_LOOKBACK': 20, 'VOL_MIN_AVG': 1000000, 'VOL_MIN_SURGE_RATIO': 1.3, 'MIN_CONFIRMATIONS': 2, 

,params,trades,win_rate,avg_net,total_net,max_dd,sharpe,objective
0,"{'USE_RSI': True, 'RSI_LEN': 21, 'RSI_LONG_MIN...",6356,0.610761,1199.954577,7.626911e+06,21870.323737,0.213065,7.626911e+06
1,"{'USE_RSI': True, 'RSI_LEN': 21, 'RSI_LONG_MIN...",6076,0.608295,1253.239238,7.614682e+06,25738.148036,0.213145,7.614682e+06
2,"{'USE_RSI': True, 'RSI_LEN': 21, 'RSI_LONG_MIN...",6338,0.609656,1199.391603,7.601744e+06,21870.323737,0.212664,7.601744e+06
3,"{'USE_RSI': True, 'RSI_LEN': 21, 'RSI_LONG_MIN...",6058,0.607131,1252.808567,7.589514e+06,25738.148036,0.212757,7.589514e+06
4,"{'USE_RSI': True, 'RSI_LEN': 21, 'RSI_LONG_MIN...",5936,0.615903,1273.149043,7.557413e+06,25738.148036,0.217748,7.557413e+06
5,"{'USE_RSI': True, 'RSI_LEN': 21, 'RSI_LONG_MIN...",6294,0.608516,1200.456353,7.555672e+06,22268.975691,0.212346,7.555672e+06
6,"{'USE_RSI': True, 'RSI_LEN': 21, 'RSI_LONG_MIN...",6008,0.608522,1254.490499,7.536979e+06,26136.799990,0.212313,7.536979e+06
7,"{'USE_RSI': True, 'RSI_LEN': 21, 'RSI_LONG_MIN...",5918,0.614735,1272.768740,7.532245e+06,25738.148036,0.217354,7.532245e+06
8,"{'USE_RSI': True, 'RSI_LEN': 21, 'RSI_LONG_MIN...",6276,0.607393,1199.889257,7.530505e+06,22268.975691,0.211943,7.530505e+06
9,"{'USE_RSI': True, 'RSI_LEN': 21, 'RSI_LONG_MIN...",5990,0.607346,1254.058698,7.511812e+06,26136.799990,0.211923,7.511812e+06


[oos] Backtesting with best params: {'USE_RSI': True, 'RSI_LEN': 21, 'RSI_LONG_MIN': 50, 'RSI_SHORT_MAX': 50, 'USE_MACD': True, 'MACD_FAST': 12, 'MACD_SLOW': 26, 'MACD_SIGNAL': 9, 'MACD_MODE': 'hist_above0', 'USE_ADX': True, 'ADX_LEN': 14, 'ADX_MIN': 18, 'USE_SMA': True, 'SMA_FAST': 20, 'SMA_SLOW': 50, 'USE_BB': True, 'BB_LEN': 20, 'BB_STD': 2.0, 'BB_SQUEEZE_PCTL': 10, 'USE_VOLUME': True, 'VOL_LOOKBACK': 20, 'VOL_MIN_AVG': 1000000, 'VOL_MIN_SURGE_RATIO': 1.3, 'MIN_CONFIRMATIONS': 2, 'MAX_HOLD': 7}
OOS Metrics: {'trades': 9876, 'win_rate': np.float64(0.5937626569461321), 'avg_net': 748.6704066872671, 'total_net': 7393868.93644345, 'max_dd': 17512.889395912178, 'sharpe': 0.28364027959573807}


,ticker,signal_dt,entry_dt,entry_time,exit_dt,exit_time,entry_date,exit_date,pattern_detected,exit_reason,...,static_stop,gross_pnl,fees,net_pnl,reason,signal_bar,patterns,indicator_conf_true,indicator_values,stop_levels
0,BDL.NS,2018-04-26 00:00:00,2018-04-27 00:00:00,00:00:00,2018-04-30 00:00:00,00:00:00,2018-04-27,2018-04-30,INVERTED_HAMMER_BULL,SL_SMA_TRAIL_20,...,165.606595,-713.006049,60.186840,-773.192889,SL_SMA_TRAIL_20,2018-04-26,INVERTED_HAMMER_BULL,,,
1,BDL.NS,2018-05-03 00:00:00,2018-05-04 00:00:00,00:00:00,2018-05-04 00:00:00,00:00:00,2018-05-04,2018-05-04,"HARAMI_CROSS_BULL,HAMMER_BULL",SL_SMA_TRAIL_20,...,161.028456,525.457585,61.427069,464.030516,SL_SMA_TRAIL_20,2018-05-03,"HARAMI_CROSS_BULL,HAMMER_BULL",,,
2,BDL.NS,2018-05-10 00:00:00,2018-05-11 00:00:00,00:00:00,2018-05-11 00:00:00,00:00:00,2018-05-11,2018-05-11,INVERTED_HAMMER_BULL,SL_SMA_TRAIL_20,...,161.578477,1190.701328,62.080534,1128.620794,SL_SMA_TRAIL_20,2018-05-10,INVERTED_HAMMER_BULL,,,
3,BDL.NS,2018-05-14 00:00:00,2018-05-15 00:00:00,00:00:00,2018-05-15 00:00:00,00:00:00,2018-05-15,2018-05-15,MORNING_STAR_BULL,SL_SMA_TRAIL_20,...,158.357277,2195.898875,63.129555,2132.769320,SL_SMA_TRAIL_20,2018-05-14,MORNING_STAR_BULL,,,
4,BDL.NS,2018-06-06 00:00:00,2018-06-07 00:00:00,00:00:00,2018-06-07 00:00:00,00:00:00,2018-06-07,2018-06-07,HAMMER_BULL,SL_SMA_TRAIL_20,...,159.005793,361.453786,61.259805,300.193981,SL_SMA_TRAIL_20,2018-06-06,HAMMER_BULL,,,


Chunk6 loaded
